# Referee Report Major Comment #2
One of the reasons for spectroscopically-derived SFR is to obtain a cleaner separation between SF and quiescent galaxies at z~0 than a simple color cut. However, the quiescent fractions at higher redshift taken from Tinker et al. 2013 were obtained using NUV-R and R-J color-color cut. Would these different classification methods at different redshifts affect the results? The authors could separate the SF and quiescent galaxies at z~0 using a color cut similar to that in Tinker et al. 2013 to check this.

In [1]:
import numpy as np
import h5py
import pickle
from pydl.pydlutils.spheregroup import spherematch

import matplotlib.pyplot as plt
%matplotlib inline 

from ChangTools.fitstables import mrdfits
from ChangTools.plotting import prettyplot
from ChangTools.plotting import prettycolors

## import relevant data

In [2]:
# local directory
local_dir = '/Volumes/ChangHDD/project/cenque/observations/'

In [3]:
# VAGC group catalog (all) M_r cut = 18
vagc_file = ''.join([local_dir, 'GroupCat.Mr18.Mass9.4.D360.all.hdf5']) 
f = h5py.File(vagc_file, 'r')
grp = f['data']

# save to dictionary
vagc = {}
for key in grp.keys(): 
    vagc[key] = grp[key].value

In [4]:
# import NSA catalog that's cross matched with VAGC-GALEX-2MASS data 
# and also contains SFR derived from GALEX UV (see nsa_galex_2mass.ipynb)
nsa_file = ''.join([local_dir, 'NSA_2MASS_UVSFR.p'])
nsa_dat = pickle.load(open(nsa_file, 'rb'))

In [5]:
# transfer to dictionary with limited keys for convenience
nsa = {} 
for key in ['ra', 'dec', 'z']: 
    nsa[key] = nsa_dat[key]
    
nsa['mass'] = np.log10(nsa_dat['mass']) # stellar mass from k-correct

for key in nsa_dat.keys(): # SFRs that were derived from GALEX UV
    if 'sfr' in key: 
        nsa[key] = np.log10(nsa_dat[key])

# calculate some colors that will be used to address the ref report
nsa['nuv-r'] = nsa_dat['absmag'][:,1] - nsa_dat['absmag'][:,4]
nsa['r-j'] = nsa_dat['vagc_absmag_r'] - nsa_dat['vagc_absmag_j']

SyntaxError: invalid syntax (<ipython-input-5-b742b2c068db>, line 8)

In [6]:
for i in range(11): 
    print nsa_dat['absmag'][i,4], nsa_dat['vagc_absmag_r'][i]

-19.667 -19.5226287842
-18.4002 -18.4193000793
-15.5081 -14.8116579056
-20.4006 -20.5390167236
-16.5895 -16.5115909576
-15.2733 -999.0
-16.2422 -14.7895030975
-18.7064 -18.5783786774
-16.3601 -16.1415405273
-19.6146 -19.5316905975
-19.2393 -19.2129268646
